In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

## Steps
1. Join the kaggle contest - https://www.kaggle.com/competitions/planttraits2024
2. Install kaggle cli - https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md
3. Download the data - `kaggle competitions download -c planttraits2024`
4. Unzip the data
5. Install FGVC repo - `pip install -e .` and `pip install -r requirement.txt` in the desired env
5. Setup data from `data_setup.ipynb`
5. Train the model

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import omegaconf
import hydra
from lightning import Callback, LightningDataModule, LightningModule, Trainer
from omegaconf import DictConfig
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

### DataFrame Analysis

In [ ]:
# create this dataframe from data_setup.ipynb
df_all = pd.read_csv('/home/ubuntu/FGVC11/data/PlantTrait/df_processed.csv')
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/data/plant_traits_data.yaml")
datamodule: LightningDataModule = hydra.utils.instantiate(cfg)
datamodule.setup()

In [ ]:
df_all.split.value_counts()

In [ ]:
#all columns must be identical to be consider the same species
label_col = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
aux_col = list(
            map(lambda x: x.replace("mean", "sd"), label_col)
        )
metadata_col = list(set(df_all.columns) - set(label_col) - set(aux_col) - set(["id", "path", "split", "species"]))
len(label_col), len(aux_col), len(metadata_col)

In [ ]:
df_all[label_col][df_all.split != "test"].hist(bins=50, figsize=(10, 10))
plt.show()

### Adding CLIP feats

In [ ]:
# import torch
# import clip
# from PIL import Image
# from tqdm import tqdm
# import torchvision.transforms as transforms

# device = "cuda" if torch.cuda.is_available() else "cpu"
# # model, preprocess = clip.load("ViT-B/32", device=device)
# model = torch.hub.load(
#             "facebookresearch/dinov2", "dinov2_vits14_reg", pretrained=True
#         )
# model.to(device);

# # Define the transformations
# transform = transforms.Compose([
#     transforms.Resize((490, 490)),  # Resize the image to a fixed size
#     transforms.ToTensor(),  # Convert the image to a tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
# ])

# feats = []
# for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
#     image = transform(Image.open(row["path"])).unsqueeze(0).to(device)
#     with torch.no_grad():
#         image_features = model(image.to(device))
#         feats.append(image_features.view(-1).cpu().numpy())
#     # break

# feats = np.array(feats)
# # np.save("feats.npy", feats)  

In [ ]:
# np.save("feats_processed.npy", feats)  

In [ ]:
feats = np.load("feats_processed.npy")
feats.shape

In [ ]:
feat_df = pd.DataFrame(feats)

### Run XGBoost

In [ ]:
# create the training data
X = df_all[metadata_col][df_all.split != "test"].reset_index(drop=True)
X.shape

In [ ]:
# add clip feaures and save the final test data
x_final = df_all[metadata_col][df_all.split == "test"].reset_index(drop=True).values
x_final = np.concatenate([x_final, feat_df[df_all.split == "test"].values], axis=1)

dx_final = xgb.DMatrix(x_final, enable_categorical=False)
# Create a DataFrame with the corresponding IDs
submission_df = pd.DataFrame({
    'id': df_all["id"][df_all.split == "test"].values,
})
x_final.shape

In [ ]:
# load LabelEncoder for trasnforming label to log + standardization 
from fgvc.models.plant_traits_model import LabelEncoder
le = LabelEncoder()

In [ ]:
# trasnform labels using label encoder
Y = le.transform(torch.tensor(df_all[label_col][df_all.split != "test"].values)).numpy()
Y = pd.DataFrame(Y, columns=label_col)
Y.hist(bins=50, figsize=(10, 8))
plt.show()

### Train model for all the traits simultaneously

In [ ]:
# Split the data
X_train  = X[df_all.split == "train"].values
X_val = X[df_all.split == "val"].values
y_train = Y[df_all.split == "train"].values
y_val = Y[df_all.split == "val"].values

# concatenate clip features
X_train = np.concatenate([X_train, feat_df[df_all.split == "train"].values], axis=1)
X_val = np.concatenate([X_val, feat_df[df_all.split == "val"].values], axis=1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=False)
dval_reg = xgb.DMatrix(X_val, y_val, enable_categorical=False)

# set hyperparams
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", "max_depth":2}
n = 1000

evals = [(dval_reg, "validation")]

model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    evals=evals,
    verbose_eval=20,
    # Activate early stopping
    early_stopping_rounds=10
    )

print("*"*100)
# get the encoded prediction
y_train_pred = model.predict(dtrain_reg)
# decode raw predicted traits
y_train_pred = le.inverse_transform(torch.Tensor(y_train_pred)).numpy()
# get the raw labels
y_train = le.inverse_transform(torch.Tensor(y_train)).numpy()
# evaluate
print(f"R2 train: {r2_score(y_train, y_train_pred)}")

# get the encoded prediction
y_val_pred = model.predict(dval_reg)
# decode raw predicted traits
y_train_pred = le.inverse_transform(torch.Tensor(y_train_pred)).numpy()
# get the raw labels
y_train = le.inverse_transform(torch.Tensor(y_train)).numpy()
# evaluate
print(f"R2 val : {r2_score(y_val, y_val_pred)}")

sub_cols = [i.replace("_mean", "") for i in label_col]
# get the encoded prediction
y_final = model.predict(dx_final)
# decode raw predicted traits
y_final = le.inverse_transform(torch.Tensor(y_final)).numpy()
submission_df[sub_cols] = y_final
    
# compare with a good submission from kaggle to sanity check
sub = pd.read_csv('submission_good.csv')
print("R2 with good", r2_score(sub[sub.columns[1:]], submission_df[sub.columns[1:]]))
print("*"*100)


#### Map to the best trait

In [ ]:
submission_df = pd.read_csv('submission_good.csv')

In [ ]:
species_traits = df_all[label_col + ['species']][df_all.split != 'test'].groupby('species').mean()
species_traits.reset_index(inplace=True)
species_traits.head()

In [ ]:
mapped_traits = []
for _, row in tqdm(submission_df.iterrows(), total=len(submission_df)):
    obj_traits = []
    for col in label_col:
        col_sub = col.replace("_mean", "")
        pred_trt = row[col_sub]
        tgt_trts = species_traits[col]
        mapped_trait = tgt_trts[np.argmin(abs(pred_trt - tgt_trts))]
        obj_traits.append(mapped_trait)
        # break
    obj_traits = np.array(obj_traits)
    mapped_traits.append(obj_traits)

mapped_traits = np.array(mapped_traits)

# Create a DataFrame with the predictions and corresponding IDs
mapped_sub = pd.DataFrame({
    'id': submission_df['id'].values,
    'X4': mapped_traits[:, 0],
    'X11': mapped_traits[:, 1],
    'X18': mapped_traits[:, 2],
    'X50': mapped_traits[:, 3],
    'X26': mapped_traits[:, 4],
    'X3112': mapped_traits[:, 5],
})

In [ ]:
# sanity check before mapping
for col in label_col:
    col = col.replace("_mean", "")
    print(f"r2 {col}: {r2_score(sub[col], submission_df[col])}")

In [ ]:
# sanity check after mapping
for col in label_col:
    col = col.replace("_mean", "")
    print(f"r2 {col}: {r2_score(sub[col], mapped_sub[col])}")

In [ ]:
submission_df[sub.columns].to_csv("submission.csv", index=False)

In [ ]:
!kaggle competitions submit -c planttraits2024 -f submission_good.csv -m "good"

### For each trait separately

In [ ]:
# # Split the data
# X_train  = X[df_all.split == "train"].values
# X_val = X[df_all.split == "val"].values
# y_train = Y[df_all.split == "train"].values
# y_val = Y[df_all.split == "val"].values

# # concatenate clip features
# X_train = np.concatenate([X_train, feat_df[df_all.split == "train"].values], axis=1)
# X_val = np.concatenate([X_val, feat_df[df_all.split == "val"].values], axis=1)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# for idx, col in enumerate(label_col):
#     # Create regression matrices
#     dtrain_reg = xgb.DMatrix(X_train, y_train[:, idx], enable_categorical=False)
#     dval_reg = xgb.DMatrix(X_val, y_val[:, idx], enable_categorical=False)
    
#     # set hyperparams
#     params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", "max_depth":2}
#     n = 1000

#     evals = [(dval_reg, "validation")]

#     model = xgb.train(
#         params=params,
#         dtrain=dtrain_reg,
#         num_boost_round=n,
#         evals=evals,
#         verbose_eval=10,
#         # Activate early stopping
#         early_stopping_rounds=10
#         )
    
#     print("*"*100)
#     y_train_pred = model.predict(dtrain_reg)
#     y_train_pred = np.exp(SCALER.inverse_transform(y_train_pred.reshape(-1, 1)))
#     y_train = np.exp(SCALER.inverse_transform(y_train.reshape(-1, 1)))
#     print(f"R2 train {col}: {r2_score(y_train, y_train_pred)}")
    
#     y_val_pred = model.predict(dval_reg)
#     y_val_pred = np.exp(SCALER.inverse_transform(y_val_pred.reshape(-1, 1)))
#     y_val = np.exp(SCALER.inverse_transform(y_val.reshape(-1, 1)))
#     print(f"R2 val {col} : {r2_score(y_val, y_val_pred)}")


#     y_final = model.predict(dx_final)
#     y_final = np.exp(SCALER.inverse_transform(y_final.reshape(-1, 1)))
#     submission_df[col.replace("_mean", "")] = y_final
#     print("*"*100)    
#     # break
# # compare with a good submission from kaggle to sanity check
# sub = pd.read_csv('submission_good.csv')
# r2_score(sub[sub.columns[1:]], submission_df[sub.columns[1:]])

In [ ]:
# make sure columns are in right order, otherwise you'll get absurd values
# submission_df[sub.columns].to_csv('submission.csv', index=False)